# English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from src.flarepy_EK_Dra import FlareDetector_EK_Dra
from src.base_flare_detector import make_rotation_frequency_grid
"""EK Dra English translation pending 1 English translation pending process_data=True English translation pending、English translation pending。"""





DATA_EK = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
print("DATA_EK:", DATA_EK, "exists:", DATA_EK.exists())

fits_files_ek = sorted(DATA_EK.glob("*.fits")) if DATA_EK.exists() else []
print("found", len(fits_files_ek), "EK Dra FITS files")

example_ek = fits_files_ek[0] if fits_files_ek else None
print("example_ek:", example_ek)

if example_ek is None:
    detector_ek = None
    print("EK Dra English translation pending FITS English translation pending、English translation pending。")
else:
    detector_ek = FlareDetector_EK_Dra(
        file=str(example_ek),
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )
    print("rotation period per [day]:", detector_ek.per)
    print("rotation period error per_err [day]:", detector_ek.per_err)
    print("brightness_variation_amplitude:", detector_ek.brightness_variation_amplitude)


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
if detector_ek is None or detector_ek.power is None:
    print("detector_ek English translation pending、rotation_period English translation pending。")
else:
    # BaseFlareDetector English translation pending、English translation pending
    frequency, periods = make_rotation_frequency_grid(
        period_min=detector_ek.rotation_period_min,
        period_max=detector_ek.rotation_period_max,
        n_points=detector_ek.rotation_n_points,
    )
    power = detector_ek.power

    print("len(periods):", len(periods))
    print("len(power):", len(power))

    fig = px.line(
        x=periods,
        y=power,
        labels={"x": "Period [day]", "y": "Lomb–Scargle power"},
        title="03-1: Lomb–Scargle power vs period (EK Dra)",
    )
    # English translation pending
    fig.update_xaxes(autorange="reversed")

    # English translation pending
    fig.add_vline(
        x=detector_ek.per,
        line_color="red",
        line_dash="dash",
        annotation_text="per",
        annotation_position="top",
    )

    fig.show()


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
if detector_ek is None:
    print("detector_ek English translation pending。English translation pending。")
else:
    # English translation pending flux_diff English translation pending（process_data English translation pending）
    detector_ek.flux_diff()

    print("brightness_variation_amplitude:", detector_ek.brightness_variation_amplitude)
    print("starspot (area-like quantity):", detector_ek.starspot)
    print("starspot_ratio:", detector_ek.starspot_ratio)


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DS = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_V889 = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DATA_DS:", DATA_DS, "exists:", DATA_DS.exists())
print("DATA_V889:", DATA_V889, "exists:", DATA_V889.exists())

star_rows = []

# EK Dra (English translation pending detector_ek English translation pending)
if detector_ek is not None:
    flare_ratio_ek = (
        detector_ek.flare_number / detector_ek.precise_obs_time
        if detector_ek.precise_obs_time > 0
        else np.nan
    )
    star_rows.append(
        {
            "star": "EK_Dra",
            "per": detector_ek.per,
            "per_err": detector_ek.per_err,
            "amp": detector_ek.brightness_variation_amplitude,
            "flare_ratio": flare_ratio_ek,
        }
    )

# DS Tuc A
if DATA_DS.exists():
    ds_files = sorted(DATA_DS.glob("*.fits"))
    if ds_files:
        ds_example = ds_files[0]
        ds_det = FlareDetector_DS_Tuc_A(
            file=str(ds_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        flare_ratio_ds = (
            ds_det.flare_number / ds_det.precise_obs_time
            if ds_det.precise_obs_time > 0
            else np.nan
        )
        star_rows.append(
            {
                "star": "DS_Tuc_A",
                "per": ds_det.per,
                "per_err": ds_det.per_err,
                "amp": ds_det.brightness_variation_amplitude,
                "flare_ratio": flare_ratio_ds,
            }
        )

# V889 Her
if DATA_V889.exists():
    v889_files = sorted(DATA_V889.glob("*.fits"))
    if v889_files:
        v889_example = v889_files[0]
        v889_det = FlareDetector_V889_Her(
            file=str(v889_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        flare_ratio_v889 = (
            v889_det.flare_number / v889_det.precise_obs_time
            if v889_det.precise_obs_time > 0
            else np.nan
        )
        star_rows.append(
            {
                "star": "V889_Her",
                "per": v889_det.per,
                "per_err": v889_det.per_err,
                "amp": v889_det.brightness_variation_amplitude,
                "flare_ratio": flare_ratio_v889,
            }
        )

if not star_rows:
    print("English translation pending、English translation pending。")
else:
    df_star = pd.DataFrame(star_rows)
    df_star


In [ ]:
if "df_star" not in globals():
    print("English translation pending df_star English translation pending。")
else:
    fig_scatter = px.scatter(
        df_star,
        x="per",
        y="flare_ratio",
        text="star",
        labels={"per": "Rotation period [day]", "flare_ratio": "Flare rate [1/day]"},
        title="05: Rotation period vs flare rate (per file)",
    )
    fig_scatter.update_traces(textposition="top center")
    fig_scatter.show()
